In [ ]:
import tensorflow as tf
# Getting GPU device name.
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch
# If a GPU is available
if torch.cuda.is_available():    
    #set device to GPU   
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If no GPU is available
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
#Importing necessary libraries
!pip install transformers

import re
import scipy
import pandas         as pd
import io
import numpy          as np
import copy
import seaborn        as sns

import transformers
from transformers                     import  RobertaModel, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
import torch



from sklearn.metrics                  import classification_report
from sklearn.metrics                  import confusion_matrix
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.utils                    import class_weight

from torch                            import nn, optim
from torch.utils                      import data
from sklearn.decomposition            import PCA

#Seeding for deterministic results
RANDOM_SEED = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED) 
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False

CLASS_NAMES = ['MajorClaim','Claim','Premise']
# CLASS_NAMES =['Non-ADU','ADU']

MAX_LENGTH = 200                                    
BATCH_SIZE = 4
EPOCHS = 8
HIDDEN_UNITS = 128

tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-large')  #Use roberta-large or roberta-base

     |████████████████████████████████| 4.0 MB 8.0 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.5 MB 50.5 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [ ]:
#Converting labels to numbers
def label_to_int(label):
    if label == 1:
        return 0
    elif label == 2:
        return 1
    elif label == 3:
        return 2

In [ ]:
def processStanceData(df):
                                                   #Concatenating twitter and reddit data
    result1  = df.replace(np.nan, '', regex=True)                               #Getting rid of NaN values

    result1['labelValue'] = result1.ADU_Type.apply(label_to_int)     
    result1['TextSrcInre']    = result1['Text']
    result1['Features']    = result1['Sentence_Label'].str.cat(result1['Paragraph_Label'],sep=" ") 
    data = result1[['Text','Topic','TextSrcInre','Para_No','ADU_Type','labelValue','Features']].copy()    


#      replyText           - the reply post (whose stance towards the target needs to be learnt)
#      replyTextId         - the ID of the reply post
#      previousText        - the text to which replyText was replied
#      sourceText          - the source post of the conversation thread
#      label               - the label value assigned to each post
#      previoysPlusSrctext - the concatenation of the previousText and the sourceText
#      labelValue          - the numberic value assigned to each label

    data.columns = ['Text','Topic','TextSrcInre','Para_No','ADU_Type','labelValue','Features']
    return data

In [ ]:
# Reading data from AAE premise and claims file as dataFrames

# Reading data from AAE premise and claims file as dataFrames
from google.colab import drive
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/Colab Notebooks/1. ADU_Classification/Data/" 
# df_temp=pd.read_excel(path,sheet_name='data_AAE_CMV')
# # df_temp=pd.read_excel(path,sheet_name='data')
# df=df_temp.loc[df_temp['ADU_Type']!='None']
# df = df.reset_index(drop=True)
# processed_df = processStanceData(df)
# from sklearn.model_selection import train_test_split 

# x, testDf, y, y_test = train_test_split (processed_df,processed_df.labelValue.values, test_size=0.15, train_size=0.85 )
# trainDf, devDf, y_train, y_dev = train_test_split(x,y,test_size = 0.17, train_size =0.83)


# # plot_df=df['ADU_Type'].value_counts()
# # plot_df.plot(kind='bar')

Mounted at /content/gdrive


In [ ]:
# FINAl approach 


# trainDf=pd.read_csv(path+'model1_train_CMV+AAE.csv',sep="\t",index_col=False)
# trainDf=trainDf.loc[trainDf.labelValue!=0]
# trainDf=trainDf.drop(columns='Unnamed: 0')
# devDf=pd.read_csv(path+'model1_dev_CMV+AAE.csv',sep="\t",index_col=False)
# devDf=devDf.drop(columns='Unnamed: 0')
# devDf=devDf.loc[devDf.labelValue!=0]
# testDf=pd.read_csv(path+'model1_test_CMV+AAE.csv',sep="\t",index_col=False)
# testDf=testDf.drop(columns='Unnamed: 0')
# testDf=testDf.loc[testDf.labelValue!=0]

# trainDf.labelValue=trainDf.labelValue.apply(label_to_int)
# devDf.labelValue=devDf.labelValue.apply(label_to_int)
# testDf.labelValue=testDf.labelValue.apply(label_to_int)

########----------- Model 1 modified  train cmv+aae test on AAE 
# trainDf=pd.read_csv(path+'model1_train_CMV+AAE.csv',sep="\t",index_col=False)
# trainDf=trainDf.loc[trainDf.labelValue!=0]
# trainDf=trainDf.drop(columns='Unnamed: 0')
# devDf=pd.read_csv(path+'model1_dev_CMV+AAE.csv',sep="\t",index_col=False)
# devDf=devDf.drop(columns='Unnamed: 0')
# devDf=devDf.loc[devDf.labelValue!=0]
# testDf=pd.read_csv(path+'model_2_3_testDf_aae.csv',sep="\t",index_col=False)
# testDf=testDf.drop(columns='Unnamed: 0')
# testDf=testDf.loc[testDf.labelValue!=0]

# trainDf.labelValue=trainDf.labelValue.apply(label_to_int)
# devDf.labelValue=devDf.labelValue.apply(label_to_int)
# testDf.labelValue=testDf.labelValue.apply(label_to_int)

#----------- Model 2  training data is CMV+AAE , test data is AAE 

# trainDf=pd.read_csv(path+'model_2_train_CMV+AAE.csv',sep="\t",index_col=False)
# trainDf=trainDf.loc[trainDf.labelValue!=0]
# trainDf=trainDf.drop(columns='Unnamed: 0')
# devDf=pd.read_csv(path+'model_2_dev_CMV+AAE.csv',sep="\t",index_col=False)
# devDf=devDf.drop(columns='Unnamed: 0')
# devDf=devDf.loc[devDf.labelValue!=0]
# testDf=pd.read_csv(path+'model_2_3_testDf_aae.csv',sep="\t",index_col=False)
# testDf=testDf.drop(columns='Unnamed: 0')
# testDf=testDf.loc[testDf.labelValue!=0]

# trainDf.labelValue=trainDf.labelValue.apply(label_to_int)
# devDf.labelValue=devDf.labelValue.apply(label_to_int)
# testDf.labelValue=testDf.labelValue.apply(label_to_int)

###-------------- MODEL 3 AAE Baseline 

trainDf=pd.read_csv(path+'model3_trainDf_aae.csv',sep="\t",index_col=False)
trainDf=trainDf.loc[trainDf.labelValue!=0]
trainDf=trainDf.drop(columns='Unnamed: 0')
devDf=pd.read_csv(path+'model3_devDf_aae.csv',sep="\t",index_col=False)
devDf=devDf.drop(columns='Unnamed: 0')
devDf=devDf.loc[devDf.labelValue!=0]
testDf=pd.read_csv(path+'model_2_3_testDf_aae.csv',sep="\t",index_col=False)
testDf=testDf.drop(columns='Unnamed: 0')
testDf=testDf.loc[testDf.labelValue!=0]

trainDf.labelValue=trainDf.labelValue.apply(label_to_int)
devDf.labelValue=devDf.labelValue.apply(label_to_int)
testDf.labelValue=testDf.labelValue.apply(label_to_int)

In [ ]:
trainDf.labelValue.value_counts()

2    2541
1     998
0     523
Name: labelValue, dtype: int64

In [ ]:
cmv=trainDf.loc[trainDf.Para_No.isnull()]
cmv.labelValue.value_counts()

Series([], Name: labelValue, dtype: int64)

In [ ]:
devDf.labelValue.value_counts()

2    434
1    175
0     97
Name: labelValue, dtype: int64

In [ ]:
cmv_dev=devDf.loc[devDf.Para_No.isnull()]
print(cmv_dev.labelValue.value_counts())
aae_dev=devDf.loc[~devDf.Para_No.isnull()]
print(aae_dev.labelValue.value_counts())

Series([], Name: labelValue, dtype: int64)
2    434
1    175
0     97
Name: labelValue, dtype: int64


In [ ]:
testDf.labelValue.value_counts()

2    450
1    190
0     80
Name: labelValue, dtype: int64

In [ ]:
from numpy.core.arrayprint import printoptions
test_cmv=testDf.loc[testDf.Para_No.isnull()]
print(test_cmv.labelValue.value_counts())
test_aae=testDf.loc[~testDf.Para_No.isnull()]
print(test_aae.labelValue.value_counts())

Series([], Name: labelValue, dtype: int64)
2    450
1    190
0     80
Name: labelValue, dtype: int64


In [ ]:
print(len(trainDf), len(devDf),len(testDf))
print(trainDf.labelValue.value_counts(), devDf.labelValue.value_counts(),testDf.labelValue.value_counts())

4062 706 720
2    2541
1     998
0     523
Name: labelValue, dtype: int64 2    434
1    175
0     97
Name: labelValue, dtype: int64 2    450
1    190
0     80
Name: labelValue, dtype: int64


In [ ]:
#Creates a dataset which will be used to feed to RoBERTa
class StanceDataset(data.Dataset):
  def __init__(self,TextSrcInre,Features, labelValue,  tokenizer, max_len):

    # def __init__(self,firstSeq,TextSrcInre, labelValue,  tokenizer, max_len):
#     def __init__(self, firstSeq, secondSeq, TextSrcInre, labelValue,  tokenizer, max_len):
        # self.firstSeq    = firstSeq      #First input sequence that will be supplied to RoBERTa
        # self.secondSeq   = secondSeq     #Second input sequence that will be supplied to RoBERTa
        self.TextSrcInre = TextSrcInre   #Concatenation of reply+ previous+ src text to get features from 1 training example
        self.Features = Features
        self.labelValue  = labelValue    #label value for each training example in the dataset
        self.tokenizer   = tokenizer     #tokenizer that will be used to tokenize input sequences (Uses BERT-tokenizer here)
        self.max_len     = max_len       #Maximum length of the tokens from the input sequence that BERT needs to attend to

  def __len__(self):
        return len(self.labelValue)

  def __getitem__(self, item):
        # firstSeq    = str(self.firstSeq[item])
        # secondSeq   = str(self.secondSeq[item])
        TextSrcInre = str(self.TextSrcInre[item])
        Features = str(self.Features[item])

    #Encoding the first and the second sequence to a form accepted by RoBERTa
    #RoBERTa does not use token_type_ids to distinguish the first sequence from the second sequnece.
        encoding = tokenizer.encode_plus(
            # firstSeq,
            # secondSeq,
            TextSrcInre,
            Features,
            max_length = self.max_len,
            add_special_tokens= True,
            truncation = True,
            pad_to_max_length = True,
            # padding=True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        return {
            # 'firstSeq' : firstSeq,
            # 'secondSeq' : secondSeq,
            'TextSrcInre': TextSrcInre,
            'Features' : Features,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labelValue'  : torch.tensor(self.labelValue[item], dtype=torch.long)
        }


In [ ]:
#Creates a data loader
def createDataLoader(dataframe, tokenizer, max_len, batch_size):
    ds = StanceDataset(
        # firstSeq    = dataframe.Topic.to_numpy(),
        # secondSeq   = dataframe.Topic.to_numpy(),
        TextSrcInre = dataframe.TextSrcInre.to_numpy(),
        Features = dataframe.Features.to_numpy(),
        labelValue  = dataframe.labelValue.to_numpy(),
        tokenizer   = tokenizer,
        max_len     = max_len
    )

    return data.DataLoader(
        ds,
        batch_size  = batch_size,
        shuffle     = True,
        num_workers = 2
    )

In [ ]:
#Creating data loader for training data
trainDataLoader        = createDataLoader(trainDf, tokenizer, MAX_LENGTH, BATCH_SIZE)

#Creating data loader for development data
developmentDataLoader  = createDataLoader(devDf, tokenizer, MAX_LENGTH, BATCH_SIZE)

#Creating data loader for test data
testDataLoader         = createDataLoader(testDf, tokenizer, MAX_LENGTH, BATCH_SIZE)

In [ ]:
#Instantiating the tf-idf vectorizer object
tfidf = TfidfVectorizer(min_df = 10, max_df = 0.5, ngram_range=(1,2))

x_train = trainDf['TextSrcInre'].tolist()
x_train_feats = tfidf.fit(x_train)
print('x_train_feats: ',x_train_feats)
print('length: ',len(x_train_feats.get_feature_names()))


x_train_transform = x_train_feats.transform(x_train)
tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(x_train_transform)).float()
print('x_train_transform.shape: ',x_train_transform.shape)


pca = PCA(n_components=128)
p = pca.fit(tfidf_transform_tensor)
# print(p.shape)
print(p)
X = p.transform(tfidf_transform_tensor)
# torch.from_numpy(X.values)
X = torch.from_numpy(X)
# tfidf_transform_tensor_pca = torch.tensor(scipy.sparse.csr_matrix.todense(X)).float()
print(X.type())
print(X.shape)
print(X)


x_train_feats:  TfidfVectorizer(max_df=0.5, min_df=10, ngram_range=(1, 2))
length:  1688


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


x_train_transform.shape:  (4062, 1688)
PCA(n_components=128)
torch.DoubleTensor
torch.Size([4062, 128])
tensor([[ 0.2431,  0.1241,  0.0140,  ..., -0.0171,  0.0123, -0.0427],
        [ 0.0186, -0.0032, -0.0425,  ..., -0.0084,  0.0667, -0.0153],
        [ 0.0784, -0.0111,  0.0710,  ...,  0.0408,  0.0496, -0.0245],
        ...,
        [ 0.0545, -0.0570,  0.0608,  ..., -0.0104,  0.0007,  0.0273],
        [-0.0748,  0.0248, -0.0360,  ..., -0.0606, -0.0292, -0.0116],
        [ 0.0711,  0.0804, -0.0443,  ...,  0.0020, -0.0127,  0.0397]],
       dtype=torch.float64)


In [ ]:
#This class defines the model that was used to pre-train a SNN on TF-IDF features
class Tfidf_Nn(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden  = nn.Linear(len(tfidf.get_feature_names()), HIDDEN_UNITS)
        # Output layer
        self.output  =  nn.Linear(HIDDEN_UNITS, 3)
        self.dropout = nn.Dropout(0.1)
        
        # Defining tanh activation and softmax output 
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        #print(x.shape)
        y = self.tanh(x)
        #print(y.shape)
        z = self.dropout(y)
        #print(z.shape)
        z = self.output(z)
        #print(z.shape)
        z = self.softmax(z)
        
        #Returning the ouputs from the hidden layer and the final output layer
        return  y, z
    

In [ ]:
snnmodel = Tfidf_Nn()

# model_save_name = 'Mlp_Adu_AAE.pt'
model_save_name = 'Mlp_Step2_model_3.pt'
path = F"/content/gdrive/My Drive/Colab Notebooks/1. ADU_Classification/{model_save_name}"

snnmodel.load_state_dict(torch.load(path))
snnmodel.eval()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Tfidf_Nn(
  (hidden): Linear(in_features=1688, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=3, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (tanh): Tanh()
  (softmax): Softmax(dim=1)
)

In [ ]:
'''This class defines the model that will be used for 
training and testing on the dataset.

Adapted from huggingFace
This RoBERTa model from huggingface outputs the last hidden states
and the pooled output by default. Pooled output is the classification 
token (1st token of the last hidden state) further processed by a Linear
layer and a Tanh activation function.

The pre-trained RoBERTa model is used as the primary model.
This class experiments with RoBERTa and its ensemble with TF-IDF features. 
roberta-only :            No ensembling. This just fine-tunes the RoBERTa model. 
                          The pooled output is passed through a linear layer and 
                          softmax function is finally used for preictions. 

roberta-tfIdf :           This model conatenates the 1st token of last-hidden layer
                          from RoBERTa with TF-IDF features. Various ways of this 
                          concatenation was experimented (using pooled output instead
                          of 1st token of last hidden layer etc)

roberta-pcaTfidf :        This model concatenates the pooled output from
                          RoBERTa with the PCA transformed vector.

roberta-preTrainedTfIdf : This model concatenates the pooled output from
                          RoBERTa with the hidden layer output from a pre-trained
                          SNN that was trained on TF-IDF features.

Used dropout to prevent over-fitting.'''

class StanceClassifier(nn.Module):

    def __init__(self,  n_classes):
        super(StanceClassifier, self).__init__()
        self.robertaModel              = RobertaModel.from_pretrained('roberta-large')    #use roberta-large or roberta-base
        self.model_TFIDF               = snnmodel                                        #Pre-trained SNN trained with TF-IDF features

        self.drop                      = nn.Dropout(p = 0.3)

        self.output                    = nn.Linear(self.robertaModel.config.hidden_size, n_classes)

        self.input_size_tfidf_only     = self.robertaModel.config.hidden_size + len(tfidf.get_feature_names())
        self.input_size_tfidf_pca      = self.robertaModel.config.hidden_size + HIDDEN_UNITS

        self.dense                     = nn.Linear( self.input_size_tfidf_only,  self.input_size_tfidf_only)
        self.out_proj                  = nn.Linear( self.input_size_tfidf_only, n_classes)
        self.out_pca                   = nn.Linear( self.input_size_tfidf_pca, n_classes)

        self.input_size_preTrain_tfidf = self.robertaModel.config.hidden_size +  HIDDEN_UNITS 
        self.out                       = nn.Linear(self.input_size_preTrain_tfidf, n_classes)

        self.softmax                   = nn.Softmax(dim = 1)

    def forward(self, input_ids, attention_mask, inputs_tfidf_feats, pca_transformed_feats, modelType):
        roberta_output     = self.robertaModel(
            input_ids      = input_ids,               #Input sequence tokens
            attention_mask = attention_mask )         #Mask to avoid performing attention on padding tokens
    #print(roberta_output[1].shape)
        if modelType   == 'roberta-only':
            pooled_output = roberta_output[1]           #Using pooled output
            output        = self.drop(pooled_output)
            output        = self.output(output)

        elif modelType == 'roberta-tfIdf':
            soutput = roberta_output[1]#---------        experimenting with pooled output 
            #soutput = roberta_output[0][:, 0, :]        #taking <s> token (equivalent to [CLS] token in BERT)
            x       = torch.cat((soutput, inputs_tfidf_feats) , dim=1)
            x       = self.drop(x)
            output  = self.out_proj(x)

        elif modelType == 'roberta-pcaTfidf':
            soutput = roberta_output[1]
            x       = torch.cat((soutput, pca_transformed_feats) , dim=1)
            x       = self.drop(x)
            output  = self.out_pca(x)

        elif modelType == 'roberta-TrainedTfIdf':
            tfidf_hidddenLayer, tfidf_output = self.model_TFIDF(inputs_tfidf_feats)
            #print(tfidf_hidddenLayer.shape)
            #print(tfidf_output.shape)

          #Conactenating pooled output from RoBERTa with the hidden layer from the pre-trained SNN using TF-IDF features. 
          #pooled_output = torch.cat((roberta_output[1], tfidf_output) , dim=1)-------- Experimenting with Output of pre-trained SNN 
            pooled_output = torch.cat((roberta_output[1], tfidf_hidddenLayer) , dim=1)
            output        = self.drop(pooled_output)
            output        = self.out(output)

        return self.softmax(output)



In [ ]:
#Instantiating a StanceClassifier object as our model and loading the model onto the GPU.
model = StanceClassifier(len(CLASS_NAMES))
model = model.to(device)
#print(model)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
y_train=trainDf.labelValue
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
# class_weights = dict(zip(np.unique(y_train), class_weights)),
class_weights

array([2.58891013, 1.35671343, 0.53286108])

In [ ]:
'''Using the same optimiser as used in BERT paper
with a different learning rate'''
optimizer = AdamW(model.parameters(), 
                  lr = 2e-6, 
                  # lr = 1e-5,
                  correct_bias= False)

totalSteps = len(trainDataLoader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps = totalSteps
)

'''Using class-weights to accomodate heavily imbalanced data. 
These weights were learnt by running several experiments using 
other weights and the weights that produced the best results have
finally been used here'''

# weights      = [1.0, 1.0, 1.0, 1.0]
weights = class_weights
classWeights = torch.FloatTensor(weights)
lossFunction = nn.CrossEntropyLoss(weight = classWeights).to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
#This function is used for training the model with 'roberta-TrainedTfIdf'. 
def train_epoch(
  model,
  dataLoader,
  lossFunction,
  optimizer,
  device,
  scheduler,
  n_examples
):
    model = model.train()
    losses = []
    correctPredictions = 0

    for d in dataLoader:
    
        input_ids              = d["input_ids"].to(device)                           #Loading input ids to GPU
        attention_mask         = d["attention_mask"].to(device)                      #Loading attention mask to GPU
        labelValues            = d["labelValue"].to(device)                          #Loading label value to GPU
        textSrcInre            = d["TextSrcInre"]
        Features               = d["Features"]                                  
        tfidf_transform        = x_train_feats.transform(textSrcInre)
        tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(tfidf_transform)).float()   
        pca_tensor             = p.transform(tfidf_transform_tensor)

        pca_tensor = torch.from_numpy(pca_tensor).float()
        pca_tensor = pca_tensor.to(device)
        tfidf_transform_tensor = tfidf_transform_tensor.to(device)

        #Getting the output from our model (Object of StanceClassification class) for train data
        outputs = model(
          input_ids             = input_ids,
          attention_mask        = attention_mask,
          inputs_tfidf_feats    = tfidf_transform_tensor,
          pca_transformed_feats = pca_tensor,
          modelType             = 'roberta-TrainedTfIdf'
        )

        #Determining the model predictions
        _, predictionIndices = torch.max(outputs, dim=1)
        loss = lossFunction(outputs, labelValues)

        #Calculating the correct predictions for accuracy
        correctPredictions += torch.sum(predictionIndices == labelValues)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(losses), correctPredictions.double() / n_examples


In [ ]:
#This function is used for evaluating the model on the development and test set
def eval_model(
    model, 
    dataLoader, 
    lossFunction,
    device,
    n_examples
    ):
    model = model.eval()
    losses = []
    correctPredictions = 0

    with torch.no_grad():
        for d in dataLoader:
            input_ids              = d["input_ids"].to(device)                          #Loading input ids to GPU
            attention_mask         = d["attention_mask"].to(device)                     #Loading attention mask to GPU
            labelValues            = d["labelValue"].to(device)                         #Loading label values to GPU
            textSrcInre            = d["TextSrcInre"]
            tfidf_transform        = x_train_feats.transform(textSrcInre)
            tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(tfidf_transform)).float()    

            pca_tensor             = p.transform(tfidf_transform_tensor)

            pca_tensor = torch.from_numpy(pca_tensor).float()
            pca_tensor = pca_tensor.to(device)
            tfidf_transform_tensor = tfidf_transform_tensor.to(device)

            #Getting the softmax output from model for dev data
            outputs = model(
            input_ids             = input_ids,
            attention_mask        = attention_mask,
            inputs_tfidf_feats    = tfidf_transform_tensor,
            pca_transformed_feats = pca_tensor,
            modelType             = 'roberta-TrainedTfIdf'
            )

            #Determining the model predictions
            _, predictionIndices = torch.max(outputs, dim=1)
            loss = lossFunction(outputs, labelValues)

            #Calculating the correct predictions for accuracy
            correctPredictions += torch.sum(predictionIndices == labelValues)
            losses.append(loss.item())

    return np.mean(losses), correctPredictions.double() / n_examples


In [ ]:
#fine tuning ROBERTa and validating it 

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}')
    trainLoss, trainAccuracy = train_epoch(
        model,
        trainDataLoader,
        lossFunction,
        optimizer,
        device,
        scheduler,
        len(trainDf)
      )
    print(f'Training loss {trainLoss} Training accuracy {trainAccuracy}')
    devLoss, devAccuracy = eval_model(
        model,
        developmentDataLoader,
        lossFunction,
        device,
        len(devDf)
      )
    print(f'Development loss {devLoss} Development accuracy {devAccuracy}')
    # from google.colab import drive
    # drive.mount('/content/gdrive')
    
    # model_save_name = f'Roberta_tfidf_{epoch}.pt'
    # path = F"/content/gdrive/My Drive/{model_save_name}" 
    # torch.save(model.state_dict(), path)
    print()
    print()

Epoch 1


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Training loss 0.8964268148649396 Training accuracy 0.7205809945839489


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Development loss 0.8973959051283066 Development accuracy 0.7337110481586403


Epoch 2


...

Development loss 0.7982070011607671 Development accuracy 0.774787535410765


Epoch 8


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Training loss 0.7156666988345581 Training accuracy 0.8552437223042837


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Development loss 0.7873816139953959 Development accuracy 0.7762039660056658




In [ ]:
#This function gets the predictions from the model after it is trained.
def get_predictions(model, data_loader):

    model = model.eval()
    review_texta = []
#     review_textb = []               #     !! Change - commented
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:

            textSrcInre                 = d["TextSrcInre"]
#             textbs                 = d["secondSeq"]
            input_ids              = d["input_ids"].to(device)
            attention_mask         = d["attention_mask"].to(device)
            labels                 = d["labelValue"].to(device)
            Features            = d["Features"]
            tfidf_transform        = tfidf.transform(textSrcInre)
            tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(tfidf_transform)).float()

            pca_tensor             =  p.transform(tfidf_transform_tensor)

            pca_tensor = torch.from_numpy(pca_tensor).float()
            pca_tensor = pca_tensor.to(device)
            tfidf_transform_tensor = tfidf_transform_tensor.to(device)

            #Getting the softmax output from model
            outputs = model(
                input_ids             = input_ids,
                attention_mask        = attention_mask,
                inputs_tfidf_feats    = tfidf_transform_tensor,
                pca_transformed_feats = pca_tensor,
                modelType             = 'roberta-TrainedTfIdf'
                )
            _, preds = torch.max(outputs, dim=1)     #Determining the model predictions

            review_texta.extend(textSrcInre)
#             review_textb.extend(textbs)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            real_values.extend(labels)
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
  
    return review_texta, predictions, prediction_probs, real_values
#    return review_texta, review_textb, predictions, prediction_probs, real_values


In [ ]:
#Getting model predictions on dev dataset
# firstSeq_dev, secondSeq_dev, yHat_dev, predProbs_dev, yTest_dev = get_predictions(
#   model,
#   developmentDataLoader
# )

firstSeq_dev, yHat_dev, predProbs_dev, yTest_dev = get_predictions(
  model,
  developmentDataLoader
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
 #Printing classification report for dev dataset (Evaluating the model on Dev set)
print(classification_report(yTest_dev, yHat_dev, target_names= CLASS_NAMES,digits=4))
print(confusion_matrix(yTest_dev, yHat_dev))

              precision    recall  f1-score   support

  MajorClaim     0.7213    0.9072    0.8037        97
       Claim     0.5607    0.5543    0.5575       175
     Premise     0.8832    0.8364    0.8592       434

    accuracy                         0.7762       706
   macro avg     0.7217    0.7660    0.7401       706
weighted avg     0.7810    0.7762    0.7768       706

[[ 88   9   0]
 [ 30  97  48]
 [  4  67 363]]


In [ ]:
#Getting model predictions on test dataset
firstSeq_test, yHat_test, predProbs_test, yTest_test = get_predictions(
  model,
  testDataLoader
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
#Printing classification report for test dataset (Evaluating the model on test set)
print(classification_report(yTest_test, yHat_test, target_names= CLASS_NAMES,digits=4))

              precision    recall  f1-score   support

  MajorClaim     0.6875    0.9625    0.8021        80
       Claim     0.5870    0.5684    0.5775       190
     Premise     0.8703    0.8200    0.8444       450

    accuracy                         0.7694       720
   macro avg     0.7149    0.7836    0.7413       720
weighted avg     0.7752    0.7694    0.7693       720



In [ ]:
print(confusion_matrix(yTest_test, yHat_test))

[[ 77   3   0]
 [ 27 108  55]
 [  8  73 369]]


**Prediction Starts from here**

In [ ]:
#Creates a dataset which will be used to feed to RoBERTa
class StanceIcleDataset(data.Dataset):
  def __init__(self,TextSrcInre,  tokenizer, max_len):

    # def __init__(self,firstSeq,TextSrcInre, labelValue,  tokenizer, max_len):
#     def __init__(self, firstSeq, secondSeq, TextSrcInre, labelValue,  tokenizer, max_len):
        # self.firstSeq    = firstSeq      #First input sequence that will be supplied to RoBERTa
        # self.secondSeq   = secondSeq     #Second input sequence that will be supplied to RoBERTa
        self.TextSrcInre = TextSrcInre   #Concatenation of reply+ previous+ src text to get features from 1 training example
        self.tokenizer   = tokenizer     #tokenizer that will be used to tokenize input sequences (Uses BERT-tokenizer here)
        self.max_len     = max_len       #Maximum length of the tokens from the input sequence that BERT needs to attend to

  def __len__(self):
        return len(self.TextSrcInre)

  def __getitem__(self, item):
        # firstSeq    = str(self.firstSeq[item])
        # secondSeq   = str(self.secondSeq[item])
        TextSrcInre = str(self.TextSrcInre[item])

    #Encoding the first and the second sequence to a form accepted by RoBERTa
    #RoBERTa does not use token_type_ids to distinguish the first sequence from the second sequnece.
        encoding = tokenizer.encode_plus(
            # firstSeq,
            # secondSeq,
            TextSrcInre,
            max_length = self.max_len,
            add_special_tokens= True,
            truncation = True,
            pad_to_max_length = True,
            # padding=True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        return {
            # 'firstSeq' : firstSeq,
            # 'secondSeq' : secondSeq,
            'TextSrcInre': TextSrcInre,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }


In [ ]:
#Creates a data loader
def createIcleDataLoader(dataframe, tokenizer, max_len, batch_size):
    ds = StanceIcleDataset(
        TextSrcInre = dataframe.Text.to_numpy(),
        # Features = dataframe.Features.to_numpy(),
        tokenizer   = tokenizer,
        max_len     = max_len
    )

    return data.DataLoader(
        ds,
        batch_size  = batch_size,
        shuffle     = False,
        num_workers = 2
    )

In [ ]:
#This function gets the predictions from the model after it is trained.
def get_icle_predictions(model, data_loader):

    model = model.eval()
    review_texta = []
#     review_textb = []               #     !! Change - commented
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:

            textSrcInre                 = d["TextSrcInre"]
#             textbs                 = d["secondSeq"]
            input_ids              = d["input_ids"].to(device)
            attention_mask         = d["attention_mask"].to(device)
            # labels                 = d["labelValue"].to(device)
            # Features            = d["Features"]
            tfidf_transform        = tfidf.transform(textSrcInre)
            tfidf_transform_tensor = torch.tensor(scipy.sparse.csr_matrix.todense(tfidf_transform)).float()

            pca_tensor             =  p.transform(tfidf_transform_tensor)

            pca_tensor = torch.from_numpy(pca_tensor).float()
            pca_tensor = pca_tensor.to(device)
            tfidf_transform_tensor = tfidf_transform_tensor.to(device)

            #Getting the softmax output from model
            outputs = model(
                input_ids             = input_ids,
                attention_mask        = attention_mask,
                inputs_tfidf_feats    = tfidf_transform_tensor,
                pca_transformed_feats = pca_tensor,
                modelType             = 'roberta-TrainedTfIdf'
                )
            _, preds = torch.max(outputs, dim=1)     #Determining the model predictions

            review_texta.extend(textSrcInre)
#             review_textb.extend(textbs)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            # real_values.extend(labels)
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    # real_values = torch.stack(real_values).cpu()
  
    return review_texta, predictions, prediction_probs
#    return review_texta, review_textb, predictions, prediction_probs, real_values


In [ ]:
def int_to_label(label):
    if label   == 0:
        return 'MajorClaim'
    elif label == 1:
        return 'Claim'
    elif label == 2:
        return 'Premise'

In [ ]:
#Creating data loader for development data
path3= F"/content/gdrive/My Drive/Colab Notebooks/1. ADU_Classification/ICLE1000_ADU_NonADU_epoch5_roberta_large.xlsx" 
icle_df=pd.read_excel(path3,sheet_name='Sheet1',usecols=['Text','Ann_Adu'])
icle_adu_df=icle_df.loc[icle_df['Ann_Adu']!='Non-ADU']
icle_adu_df['index1'] = icle_adu_df.index
ICLE_DataLoader  = createIcleDataLoader(icle_adu_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
icle_adu_df.Ann_Adu.value_counts()

ADU    26781
Name: Ann_Adu, dtype: int64

In [ ]:
firstSeq_ICLE, yHat_icle, predProbs_icle = get_icle_predictions(
  model,
  ICLE_DataLoader
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
# from google.colab import files 

index_1=pd.DataFrame(icle_adu_df['index1'])
index_1=index_1.reset_index(drop=True)
predictions=pd.DataFrame(list(zip(firstSeq_ICLE, yHat_icle.numpy())),columns=['Text','ADU_Type'])
concatenated_dataframes = pd.concat([index_1, predictions],axis=1,ignore_index=True)
concatenated_dataframes.columns=['index','Text','Ann_Adu']
concatenated_dataframes['Ann_Adu']=concatenated_dataframes.Ann_Adu.apply(int_to_label)
concatenated_dataframes.Ann_Adu.value_counts()

Premise       21332
Claim          5249
MajorClaim      200
Name: Ann_Adu, dtype: int64

In [ ]:
# icle_non_adu_df=icle_df.loc[icle_df['Ann_Adu']=='Non-ADU']
# icle_non_adu_df=icle_non_adu_df.reset_index()
# icle_non_adu_df['Ann_Adu']='None'
# icle_non_adu_df=icle_non_adu_df.reset_index(drop=True)
# icle_non_adu_df

In [ ]:
# result=concatenated_dataframes.append(icle_non_adu_df, ignore_index=True)
# result=result.sort_values(by='index')
# result

In [ ]:
# result.Ann_Adu.value_counts()

In [ ]:
# result.to_excel('Final_ADU_ICLE_5085.xlsx')